In [ ]:
import os
import openai
import toml
import panel as pn  # GUI


# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key = os.environ["OPENAI_API_KEY"]

# Load configuration from config.toml
config = toml.load('/Users/cameronhightower/Documents/Socratic_CS_Tutor/.streamlit/config.toml')

# Set the OpenAI API key from the config file
openai.api_key = config['openai']['api_key']


llm_model = "gpt-3.5-turbo-0301"

def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.7, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model=llm_model, temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

file = '/Users/cameronhightower/Documents/LoFi/docs/conventional_loans.txt'
loader = TextLoader(file_path=file)
from langchain.indexes import VectorstoreIndexCreator

# Using the vector store index creator 

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

query = "Acting as a loan officer, ask a couple of questions to determine if a buyer is eligible for a conventional loan"

num_responses = 1
responses = []
import langchain
langchain.debug = True
for i in range(num_responses):
    responses.append(index.query(query))
for response in responses:
    display(Markdown(response))
langchain.debug = False


In [ ]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool


llm = ChatOpenAI(temperature=0, model=llm_model)

@tool
def va_pamphlet(text: str) -> str:
    """Queries the VA Pamphlet \
    Used for potential buyers who are military service members, veterans, or eligible family members. \
    The input should be an empty string. \
    This function will always return a follow up question. \
    """
    loader = PyPDFLoader("/Users/cameronhightower/Documents/LoFi/docs/va_pamphlet.pdf")
    index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])
    return index.query(str)


@tool
def general_overview(text: str) -> str:
    """Queries the general overview document \
    Used for potential buyers who are military service members, veterans, or eligible family members. \
    The input should be an empty string. \
    This function will always return a follow up question. \
    """
    loader = TextLoader(file_path=file)
    index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch).from_loaders([loader])
    return index.query(str)



agent= initialize_agent(
    [va_pamphlet], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)


In [ ]:



import panel as pn  # GUI
import langchain
pn.extension()

panels = [] # collect display 



context = [ {'role':'system', 'content':"""

You are a loan officer trying to find the best loan options for a prospective home buyer.\
When the user greets you, say "Hello, I'm here to find the best home buying options for you.  Are you ready to answer some questions?" \
Ask them a series of questions one by one to determine the best 4 loan options for them.\

"""} ] 


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=600),
)

dashboard